In [11]:
# train_pinn.py
"""
Physics-Informed Neural Network (PINN) - adapted to use the same dataset,
features and physics-derived inputs as the RF pipeline you provided.

Key points:
 - CSV_PATH updated to the dataset you provided.
 - Feature list and physics columns match the "enhanced_features" used earlier.
 - Targets mapped to the dataset columns: 'transition_detected',
   'momentum_theta_m' (theta), 'delta_99_m' (delta).
 - If separate physics/blasius columns are absent, the script uses the
   same computed momentum_theta_m and delta_99_m as the physics values
   (so the physics regularizer remains active).
"""

import os
import random
import numpy as np
import pandas as pd
from collections import Counter

# Reproducibility
SEED = 42
os.environ['PYTHONHASHSEED'] = str(SEED)
random.seed(SEED)
np.random.seed(SEED)

# ML imports
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.metrics import roc_auc_score, accuracy_score, mean_squared_error, mean_absolute_error
import joblib

# TF / Keras
import tensorflow as tf
from tensorflow.keras import layers, Model, regularizers
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

# ------------------------------
# CONFIG - updated to your dataset & inputs
# ------------------------------
CSV_PATH = r"F:\RNN based Object detection and Anomaly Classification surveillance System\Hypersonic Boundary Layer Transition Prediction\generated_dataset_M4.5-12_cone_trajectory_10000.csv"

MODEL_DIR = "models"
os.makedirs(MODEL_DIR, exist_ok=True)
SCALER_PATH = os.path.join(MODEL_DIR, "scaler.joblib")
MODEL_PATH = os.path.join(MODEL_DIR, "pinn_model.h5")
OUT_PRED_CSV = os.path.join(MODEL_DIR, "test_predictions_conform.csv")

# Training hyperparameters (kept same as before)
TEST_SIZE = 0.2
VAL_SIZE = 0.15
BATCH_SIZE = 64
EPOCHS = 200
LEARNING_RATE = 1e-3
LAMBDA_PHYS = 0.5
LOSS_WEIGHTS = [1.0, 1.0, 1.0]

# Feature set: same enhanced_features used by the RF pipeline
FEATURE_COLS = [
    'init_Mach','init_altitude_m','init_velocity_m_s','Tw_over_Tinf_init',
    'cone_half_angle_deg','nose_radius_m','x_sensor_m','mass_kg','A_ref_m2','roughness_m',
    'Re_per_m','Re_x','Knudsen','delta_99_m','momentum_theta_m','Re_theta'
]

# Targets (mapped to your dataset)
TARGET_COL_TRANS = 'transition_detected'    # binary label in your CSV
TARGET_COL_THETA = 'momentum_theta_m'       # used as theta target
TARGET_COL_DELTA = 'delta_99_m'             # used as delta target

# If you have separate physics/blasius columns, you may set them here; otherwise
# we'll fall back to using the dataset's computed momentum_theta_m and delta_99_m
PHYS_THETA_COL = 'momentum_theta_m'    # physics theta (Blasius / computed)
PHYS_DELTA_COL = 'delta_99_m'          # physics delta (Blasius / computed)

# ------------------------------
# constants (same as earlier code)
# ------------------------------
R = 287.058
gamma = 1.4
mu0 = 1.716e-5
T0_ref = 273.15
S_sutherland = 110.4

DEFAULTS = {
    'cone_half_angle_deg': 10.0,
    'nose_radius_m': 0.01,
    'x_sensor_m': 1.0,
    'T_wall_K': 300.0,
    'mass_kg': 500.0,
    'A_ref_m2': np.pi * 0.01**2,
    'roughness_m': 1e-6
}

# ------------------------------
# Utilities
# ------------------------------
def atmosphere(h):
    if h < 11000.0:
        T = 288.15 - 0.0065 * h
        p = 101325.0 * (T / 288.15) ** 5.255877
    elif h < 20000.0:
        T = 216.65
        p11 = 101325.0 * (216.65 / 288.15) ** 5.255877
        p = p11 * np.exp(- (h - 11000.0) / 6341.97)
    else:
        T = 216.65
        rho0 = 0.08803
        scale_h = 7000.0
        rho = rho0 * np.exp(-(h - 20000.0)/scale_h)
        p = rho * R * T
        return T, p, rho
    rho = p / (R * T)
    return T, p, rho

def viscosity_sutherland(T):
    T = max(T, 1e-6)
    return mu0 * (T / T0_ref)**1.5 * (T0_ref + S_sutherland) / (T + S_sutherland)

# ------------------------------
# Model builder (architecture unchanged)
# ------------------------------
def build_pinn_model(input_dim, shared_units=[128, 128], weight_decay=1e-6):
    X_in = layers.Input(shape=(input_dim,), name="input_layer")
    shared = X_in
    for i, u in enumerate(shared_units):
        shared = layers.Dense(
            u,
            activation="tanh",
            kernel_regularizer=regularizers.l2(weight_decay),
            name=f"shared_dense_{i+1}"
        )(shared)
    # Probability branch
    prob = layers.Dense(64, activation="tanh", name="prob_dense_1")(shared)
    prob = layers.Dense(1, activation="sigmoid", name="probability")(prob)
    # Theta branch
    theta_h = layers.Dense(64, activation="tanh", name="theta_dense_1")(shared)
    theta_out = layers.Dense(1, activation="relu", name="theta_pred")(theta_h)
    # Delta branch
    delta_h = layers.Dense(32, activation="tanh", name="delta_dense_1")(shared)
    delta_out = layers.Dense(1, activation="relu", name="delta_pred")(delta_h)
    model = Model(inputs=X_in, outputs=[prob, theta_out, delta_out], name="PINN_Model")
    return model

# ------------------------------
# Custom physics-regularized losses (unchanged)
# ------------------------------
def build_custom_theta_loss(lambda_phys=LAMBDA_PHYS):
    def theta_loss(y_true, y_pred):
        theta_data = tf.expand_dims(y_true[:, 0], axis=-1)
        theta_phys = tf.expand_dims(y_true[:, 1], axis=-1)
        mse_data = tf.reduce_mean(tf.square(theta_data - y_pred))
        mse_phys = tf.reduce_mean(tf.square(theta_phys - y_pred))
        return mse_data + lambda_phys * mse_phys
    return theta_loss

def build_custom_delta_loss(lambda_phys=LAMBDA_PHYS):
    def delta_loss(y_true, y_pred):
        delta_data = tf.expand_dims(y_true[:, 0], axis=-1)
        delta_phys = tf.expand_dims(y_true[:, 1], axis=-1)
        mse_data = tf.reduce_mean(tf.square(delta_data - y_pred))
        mse_phys = tf.reduce_mean(tf.square(delta_phys - y_pred))
        return mse_data + lambda_phys * mse_phys
    return delta_loss

# ------------------------------
# Upsampling (kept)
# ------------------------------
def upsample_minority(X, y_prob, y_theta_with_phys, y_delta_with_phys):
    classes, counts = np.unique(y_prob.ravel(), return_counts=True)
    if len(classes) == 1:
        return X, y_prob, y_theta_with_phys, y_delta_with_phys
    idx0 = np.where(y_prob.ravel() == 0)[0]
    idx1 = np.where(y_prob.ravel() == 1)[0]
    if len(idx1) > len(idx0):
        idx0, idx1 = idx1, idx0
    n0 = len(idx0); n1 = len(idx1)
    if n1 == 0:
        return X, y_prob, y_theta_with_phys, y_delta_with_phys
    times = int(np.ceil(n0 / n1))
    idx1_upsampled = np.tile(idx1, times)[:n0]
    idx_balanced = np.concatenate([idx0, idx1_upsampled])
    np.random.shuffle(idx_balanced)
    return X[idx_balanced], y_prob[idx_balanced], y_theta_with_phys[idx_balanced], y_delta_with_phys[idx_balanced]

# ------------------------------
# Main pipeline
# ------------------------------
def main():
    print("Loading dataset:", CSV_PATH)
    df = pd.read_csv(CSV_PATH)
    print("Dataset shape:", df.shape)

    # Verify required columns exist (features and targets)
    missing_features = [c for c in FEATURE_COLS if c not in df.columns]
    if missing_features:
        print("Warning: some FEATURE_COLS missing from CSV. Those will be imputed or computed if possible:", missing_features)

    # Check targets
    for target in [TARGET_COL_TRANS, TARGET_COL_THETA, TARGET_COL_DELTA]:
        if target not in df.columns:
            raise ValueError(f"Missing required target/physics column in CSV: {target} (expected). Please check your CSV.")

    # If physics columns not present separately, use the same computed columns as physics values
    phys_theta_col = PHYS_THETA_COL if PHYS_THETA_COL in df.columns else TARGET_COL_THETA
    phys_delta_col = PHYS_DELTA_COL if PHYS_DELTA_COL in df.columns else TARGET_COL_DELTA

    # Prepare features matrix (use FEATURE_COLS order)
    X_df = df.copy()
    for f in FEATURE_COLS:
        if f not in X_df.columns:
            # create column filled with NaN so imputer can fill
            X_df[f] = np.nan

    X_use = X_df[FEATURE_COLS].astype(float)

    # Impute missing with median
    imputer = SimpleImputer(strategy="median")
    X_imputed = imputer.fit_transform(X_use)

    # Scale features
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X_imputed)
    joblib.dump(scaler, SCALER_PATH)
    print("Saved scaler to", SCALER_PATH)

    # Prepare packed targets: [data_target, physics_value]
    y_prob_raw = df[TARGET_COL_TRANS].astype(float).values.reshape(-1, 1)
    y_theta_packed = np.vstack([df[TARGET_COL_THETA].astype(float).values, df[phys_theta_col].astype(float).values]).T
    y_delta_packed = np.vstack([df[TARGET_COL_DELTA].astype(float).values, df[phys_delta_col].astype(float).values]).T

    # Train/test split with stratify on transition label
    X_train, X_test, y_prob_train, y_prob_test, y_theta_train, y_theta_test, y_delta_train, y_delta_test = train_test_split(
        X_scaled, y_prob_raw, y_theta_packed, y_delta_packed, test_size=TEST_SIZE, random_state=SEED, stratify=y_prob_raw
    )

    # Upsample minority
    X_train_bal, y_prob_train_bal, y_theta_train_bal, y_delta_train_bal = upsample_minority(
        X_train, y_prob_train, y_theta_train, y_delta_train
    )

    print("Train class distribution (after balancing):", Counter(y_prob_train_bal.ravel()))
    print("Test class distribution:", Counter(y_prob_test.ravel()))

    # Build model (same architecture)
    model = build_pinn_model(input_dim=X_train_bal.shape[1], shared_units=[128, 128], weight_decay=1e-6)
    model.summary()

    # Losses & compile
    bce_loss = tf.keras.losses.BinaryCrossentropy()
    theta_loss = build_custom_theta_loss(lambda_phys=LAMBDA_PHYS)
    delta_loss = build_custom_delta_loss(lambda_phys=LAMBDA_PHYS)
    optimizer = tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE)
    model.compile(
        optimizer=optimizer,
        loss=[bce_loss, theta_loss, delta_loss],
        loss_weights=LOSS_WEIGHTS,
        metrics={'probability': ['AUC']}
    )

    # Callbacks
    cb_early = EarlyStopping(monitor="val_loss", patience=15, restore_best_weights=True, verbose=1)
    cb_chkpt = ModelCheckpoint(filepath=os.path.join(MODEL_DIR, "best_pinn_model.h5"), save_best_only=True, monitor="val_loss", verbose=1)
    cb_reduce = ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=7, min_lr=1e-7, verbose=1)

    # Fit
    history = model.fit(
        X_train_bal,
        [y_prob_train_bal, y_theta_train_bal, y_delta_train_bal],
        validation_split=VAL_SIZE,
        epochs=EPOCHS,
        batch_size=BATCH_SIZE,
        callbacks=[cb_early, cb_chkpt, cb_reduce],
        verbose=2
    )

    # Save final model
    model.save(MODEL_PATH)
    print("Saved trained model to", MODEL_PATH)

    # Evaluate on test
    preds = model.predict(X_test, batch_size=BATCH_SIZE)
    prob_pred = preds[0].ravel()
    theta_pred = preds[1].ravel()
    delta_pred = preds[2].ravel()

    prob_true = y_prob_test.ravel()
    theta_true = y_theta_test[:, 0]
    theta_phys_test = y_theta_test[:, 1]
    delta_true = y_delta_test[:, 0]
    delta_phys_test = y_delta_test[:, 1]

    # Metrics
    try:
        auc = roc_auc_score(prob_true, prob_pred)
    except ValueError:
        auc = None
    acc = accuracy_score(prob_true, (prob_pred > 0.5).astype(int))
    theta_mse = mean_squared_error(theta_true, theta_pred)
    theta_mae = mean_absolute_error(theta_true, theta_pred)
    delta_mse = mean_squared_error(delta_true, delta_pred)
    delta_mae = mean_absolute_error(delta_true, delta_pred)
    theta_phys_dev = mean_squared_error(theta_phys_test, theta_pred)
    delta_phys_dev = mean_squared_error(delta_phys_test, delta_pred)

    print("\n--- TEST METRICS ---")
    print("AUC (prob):", auc)
    print("Accuracy (prob @0.5):", acc)
    print("Theta -> MSE: {:.6e}, MAE: {:.6e}".format(theta_mse, theta_mae))
    print("Delta -> MSE: {:.6e}, MAE: {:.6e}".format(delta_mse, delta_mae))
    print("Theta physics dev (MSE vs physics): {:.6e}".format(theta_phys_dev))
    print("Delta physics dev (MSE vs physics): {:.6e}".format(delta_phys_dev))

    # Save test predictions in a CSV with column names matching RF pipeline pattern
    out_df = pd.DataFrame({
        "transition_true": prob_true,
        "transition_pred": prob_pred,
        "momentum_theta_true": theta_true,
        "momentum_theta_pred": theta_pred,
        "momentum_theta_blasius": theta_phys_test,
        "delta_99_true": delta_true,
        "delta_99_pred": delta_pred,
        "delta_99_blasius": delta_phys_test
    })
    out_df.to_csv(OUT_PRED_CSV, index=False)
    print("Saved test predictions to", OUT_PRED_CSV)

if __name__ == "__main__":
    main()


Loading dataset: F:\RNN based Object detection and Anomaly Classification surveillance System\Hypersonic Boundary Layer Transition Prediction\generated_dataset_M4.5-12_cone_trajectory_10000.csv
Dataset shape: (10000, 27)


ValueError: Missing required target/physics column in CSV: momentum_theta_m (expected). Please check your CSV.